## Title

[Generative Adversarial Nets
](https://papers.nips.cc/paper/2014/hash/5ca3e9b122f61f8f06494c97b1afccf3-Abstract.html)

---

## Authors and Year

Ian J. Goodfellow, Jean Pouget-Abadie, Mehdi Mirza, Bing Xu, David Warde-Farley, Sherjil Ozair, Aaron Courville, Yoshua Bengio (2014)


---

## How it works?

<p align="center">
  <img src="./fig/gen_struc.png" alt="factorio thumbnail"/>
</p> 

### 게임이론과 GAN
GAN을 **게임이론**으로 해석하면 **discriminator와 generator 간의 아래와 같이 mini-max game**이라 볼수 있습니다.

$$
\min_{G}\max_{D}V(G,D)=E_{x\sim{p_d}(x)}[lnD] + E_{z\sim{p_z}(x)}[ln(1-D(G(z)))]
\\
p_{d}: dist. \ of \ data , \ p_{z}: dist.  \ of \ gen ,  \\
D: discriminator, \ G: generator
$$

- discriminator는 generator를 간파하고
- generator는 discriminator를 속여야한다.
- 이 mini-max game을 **수식으로 표현**하면 아래와 같다.
    - discriminator는 **데이터의 확률 분포**를 모사하고,
    - generator는 **난수에서 데이터 분포로 가는 맵핑**을 만들어 낸다.


위의 **목적 함수**를 utility로 해석하면 모델이 모사하는 분포는 mixed strategy가 되고 조건을 만족하는 전략(모델)은 minmax strategy이고 zero-sum game에서 mini-max strategy는 ‘내쉬균형’입니다.([min-max theorem](https://en.wikipedia.org/wiki/Min-max_theorem)) 

⇒ **즉 Adversarial network의 optimum은 "내쉬균형"입니다.**

### GAN의 학습 방법

<p align="center">
  <img src="./fig/gen_update_model.png" alt="factorio thumbnail"/>
</p> 


하지만 **내쉬균형을 찾기는 매우 어렵습니다.** 몇가지 알고리즘이 존재하나 특정 상황에 어떤 것이 잘 동작할 지는 알수가 없습니다. 때문에 논문에서는 두 player의 전략(모델)을  **gradient-based 최적화**로 **동시에 업데이트** 하는 방법을 제안합니다. 해당 방법을 통해 근사적으로 optimum을 찾아낼 수 있습니다. 허나 해당 방식은 **이론적으로 수렴을 보장 하지는 않습니다.**(한쪽을 전략이 변하면 다른쪽의 전략이 best response가 아니게 될 수 있으므로 이론적으로 optimum을 보장받을 수 없습니다.)


### Jensen-Shannon divergence로 해석한 목적함수


<p align="center">
  <img src="./fig/kldiv_viz.gif" alt="factorio thumbnail"/>
</p> 

또한 목적함수를 분포간의 거리로도 해석가능합니다. 위의 목적함수 식에서 generator에 관한 변수를 고정하고 $D^{*}$를 대입하여 보면 식은 아래와 같이 정리됩니다.

$$
\begin{align*}
for \ fixed  \ G, \\ 
 \max_{D}V(G, D)

&=E_{x \sim p_d}[lnD^*(x)] + E_{z \sim p_z}[1-lnD^*(G(z))]\\

&=E_{x \sim p_d}[lnD^*(x)] + E_{x \sim p_g}[1-lnD^*(x)]\\

&= E_{x \sim{p_d}}[\ln{\frac{p_d(x)}{p_d(x)+p_g(x)}}] + E_{x \sim{p_g}}[\ln{\frac{p_g(x)}{p_d(x)+p_g(x)}}]
\\
&= D_{KL}(p_{d}||\frac{1}{2}(p_d+p_g)) + D_{KL}(p_{g}||\frac{1}{2}(p_d+p_g)) + C  
\end{align*}
$$

정리된 식은 $p_g$와  $p_d$간의 "**Jensen-Shannon divergence**"가 됩니다. 이러한 점이 maximum likelihood model과 adversarial model간의 차이를 만듭니다.

<p align="center">
  <img src="./fig/Difference-between-maximum-likelihood-and-adversarial-learning-The-red-lines-represent.png" alt="factorio thumbnail"/>
</p> 

maximum likelihood models이 $D_{KL}(p_d||p_g)$만을 고려하는 반면에 Jensen-Shannon divergence을 최적화 하는 adversarial model은 **$D_{KL}(p_d||p_g)$와 $D_{KL}(p_g||p_d)$를 모두 고려**하게 됩니다. 이것이 의미하는 바는 아래와 같습니다. 

$D_{KL}(p_d||p_g)$로 최적화 되는 경우는 G가 D의 모든 mode를 cover하려는 방향으로 학습됩니다. 

⇒ 즉 데이터의 모든 샘플을 만들어 낼 수 있지만 쌩뚱 맞은 샘플을 만들기도 하게 됩니다.(이 부분이 maximum likelihood models의 약점입니다.)

반면에 $D_{KL}(p_g||p_d)$를 최적화 하는 경우 G는 D의 가장 큰 mode를 찾는 방향으로 학습됩니다. 

⇒ 즉 소수의 데이터 특징은 무시하고 가장 빈도가 높은 특징들을 추려서 학습하게 됩니다. 때문에 minor한 특징을 놓치게 됩니다.

즉 두 가지를 모두 고려한다는 것은 두 지표의 장점 사이의 적절한 균형을 찾는 다고 볼 수 있습니다. 이러한 경향은 아래의 그림으로 설명이 됩니다. 좌측의 그림이 원본 데이터의 mode를 최대한 커버하는 방향으로 적합된 반면, 우측의 그림은 가장 큰 mode에 집중하여 적합된 것을 알 수 있습니다.

<p align="center">
  <img src="./fig/js_by_factor.png" alt="factorio thumbnail"/>
</p> 


최종적으로 위에 언급된 식을 실제 최적화 하기위한 알고리즘은 아래와 같습니다. 아래의 알고리즘에서 discriminator는 $\frac{1}{m}\sum_{i=1}^{m}[logD(x^i)+log(1-D(G(z^i)))] $을 maximize하고 generator는 
$\frac{1}{m}\sum_{i=1}^{m}log(1-D(G(z^i)))$를 minimize하며 학습됩니다.

<p align="center">
  <img src="./fig/gan_algo.png" alt="factorio thumbnail"/>
</p> 

---

## 마치며...

GAN이 학습하는 방식에는 아래와 같은 문제점이 존재합니다.

- 첫째로 non-cooperation game에서 내쉬 균형을 찾는 것이 매우 어렵다는 점입니다.
  - 만약 Discriminator가 빠르게 학습된다면 generator가 받는 gradient값이 작아질 테고
  - 만약 Discriminator가 너무 느리게 학습된다면 generator가 의미있는 gradient를 받을 수 없습니다.
- 다른 문제로는 mode collapse 문제와 catastrophic forgetting이 있다. 


<figure class="half" align="center">
    <img src="./fig/Sample-of-100-Generated-Images-of-a-Handwritten-Number-8-at-Epoch-315-from-a-GAN-that-has-Suffered-Mode-Collapse.png">
    <img src="./fig/Line-Plots-of-Loss-and-Accuracy-for-a-Generative-Adversarial-Network-with-a-Convergence-Failure.png">
</figure>

이후의 후속 연구들은 이러한 gan의 단점을 극복함과 더불어 원하는 방향으로 생성을 유도 할 수 있는 condition mechanism을 도입하기도 합니다.


---

## 참고

Bond-Taylor, S., Leach, A., Long, Y., & Willcocks, C. G. (2021). Deep Generative Modelling: A Comparative Review of VAEs, GANs, Normalizing Flows, Energy-Based and Autoregressive Models. IEEE Transactions on Pattern Analysis and Machine Intelligence, 1–20. https://doi.org/10.1109/TPAMI.2021.3116668

Salimans, T., Goodfellow, I., Zaremba, W., Cheung, V., Radford, A., & Chen, X. (2016). Improved techniques for training GANs. Advances in Neural Information Processing Systems, 2234–2242.

Huszár, F. (2015). How (not) to Train your Generative Model: Scheduled Sampling, Likelihood, Adversary? 1–9. http://arxiv.org/abs/1511.05101